In [1]:
import nltk
import numpy as np
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle


Using TensorFlow backend.


In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?','!','.',',']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [4]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /Users/ankit/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [7]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [11]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
47/47 [==============================] - 1s 23ms/step - loss: 2.2389 - accuracy: 0.1489
Epoch 2/200
47/47 [==============================] - 0s 985us/step - loss: 2.1719 - accuracy: 0.1489
Epoch 3/200
47/47 [==============================] - 0s 750us/step - loss: 2.0619 - accuracy: 0.1915
Epoch 4/200
47/47 [==============================] - 0s 648us/step - loss: 2.0507 - accuracy: 0.1915
Epoch 5/200
47/47 [==============================] - 0s 671us/step - loss: 1.8764 - accuracy: 0.4468
Epoch 6/200
47/47 [==============================] - 0s 696us/step - loss: 1.7360 - accuracy: 0.3830
Epoch 7/200
47/47 [==============================] - 0s 667us/step - loss: 1.6968 - accuracy: 0.5319
Epoch 8/200
47/47 [==============================] - 0s 713us/step - loss: 1.5077 - accuracy: 0.5532
Epoch 9/200
47/47 [==============================] - 0s 798us/step - loss: 1.3354 - accuracy: 0.5532
Epoch 10/200
47/47 [==============================] - 0s 687us/step - loss: 1.3058 - accurac

47/47 [==============================] - 0s 1ms/step - loss: 0.0757 - accuracy: 0.9787
Epoch 82/200
47/47 [==============================] - 0s 1ms/step - loss: 0.0929 - accuracy: 0.9787
Epoch 83/200
47/47 [==============================] - 0s 920us/step - loss: 0.0425 - accuracy: 1.0000
Epoch 84/200
47/47 [==============================] - 0s 965us/step - loss: 0.0590 - accuracy: 1.0000
Epoch 85/200
47/47 [==============================] - 0s 883us/step - loss: 0.1055 - accuracy: 0.9574
Epoch 86/200
47/47 [==============================] - 0s 882us/step - loss: 0.0250 - accuracy: 1.0000
Epoch 87/200
47/47 [==============================] - 0s 910us/step - loss: 0.0812 - accuracy: 0.9787
Epoch 88/200
47/47 [==============================] - 0s 822us/step - loss: 0.0286 - accuracy: 1.0000
Epoch 89/200
47/47 [==============================] - 0s 808us/step - loss: 0.0477 - accuracy: 1.0000
Epoch 90/200
47/47 [==============================] - 0s 646us/step - loss: 0.0200 - accuracy: 1.00

Epoch 161/200
47/47 [==============================] - 0s 698us/step - loss: 0.0099 - accuracy: 1.0000
Epoch 162/200
47/47 [==============================] - 0s 674us/step - loss: 0.0053 - accuracy: 1.0000
Epoch 163/200
47/47 [==============================] - 0s 634us/step - loss: 0.0223 - accuracy: 1.0000
Epoch 164/200
47/47 [==============================] - 0s 587us/step - loss: 0.0106 - accuracy: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 628us/step - loss: 0.0376 - accuracy: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 661us/step - loss: 0.0170 - accuracy: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 637us/step - loss: 0.0198 - accuracy: 1.0000
Epoch 168/200
47/47 [==============================] - 0s 631us/step - loss: 0.0257 - accuracy: 1.0000
Epoch 169/200
47/47 [==============================] - 0s 642us/step - loss: 0.0098 - accuracy: 1.0000
Epoch 170/200
47/47 [==============================] - 0s 652us/step - lo